In [1]:
import asyncio
import httpx
import time
from typing import List
import nest_asyncio
nest_asyncio.apply()
async def fetch_data_with_session(client_id: str, session: httpx.AsyncClient) -> dict:
    """Fetch data for a specific client using the provided session"""
    try:
        response = await session.get(f"http://localhost:8000/data/{client_id}")
        response.raise_for_status()
        return response.json()
    except httpx.HTTPError as e:
        print(f"Error fetching data for client {client_id}: {e}")
        return None

async def main():
    # Configuration
    num_requests = 10
    num_clients = 3
    
    # Create a single session to be reused
    async with httpx.AsyncClient(
        timeout=30.0,
        limits=httpx.Limits(max_keepalive_connections=20, max_connections=20),
        http2=True  # Enable HTTP/2 for better performance
    ) as session:
        
        start_time = time.time()
        
        # Create tasks for all clients
        tasks = []
        for i in range(num_clients):
            client_id = f"client_{i}"
            # Create multiple requests for each client
            client_tasks = [
                fetch_data_with_session(client_id, session)
                for _ in range(num_requests)
            ]
            tasks.extend(client_tasks)
        
        # Execute all tasks concurrently
        results = await asyncio.gather(*tasks)
        
        # Filter out None results (failed requests)
        valid_results = [r for r in results if r is not None]
        
        end_time = time.time()
        total_time = end_time - start_time
        
        # Print results
        print(f"\nCompleted {len(valid_results)} requests in {total_time:.2f} seconds")
        print(f"Average time per request: {total_time/len(valid_results):.2f} seconds")
        
        # Print some sample results
        print("\nSample results:")
        for result in valid_results[:5]:
            print(result)

if __name__ == "__main__":
    # Run the async main function
    asyncio.run(main())




Completed 30 requests in 2.73 seconds
Average time per request: 0.09 seconds

Sample results:
{'client_id': 'client_0', 'value': 88, 'processing_time': 1.12}
{'client_id': 'client_0', 'value': 95, 'processing_time': 1.82}
{'client_id': 'client_0', 'value': 66, 'processing_time': 0.12}
{'client_id': 'client_0', 'value': 15, 'processing_time': 0.84}
{'client_id': 'client_0', 'value': 48, 'processing_time': 0.16}
